In [1]:
from qiskit.circuit.library import EfficientSU2
from qiskit.primitives import Estimator
from qiskit.quantum_info import SparsePauliOp
from qiskit_algorithms.minimum_eigensolvers import VQE
from qiskit_algorithms.optimizers import COBYLA

from circuit_knitting.cutting.gate_and_wire_cutting.algorithms.cut_vqe import CutVQE


# Use a list of H2 observables from the PySCF driver
H2_op_list = [('II', -1.0523732457728596),
             ('IZ', 0.39793742484317934),
             ('XX', 0.18093119978423144),
             ('ZI', -0.39793742484317934),
             ('ZZ', -0.011280104256235268)]

# Problem specification. Hamiltonian for H2 at 0.735A interatomic distance.
H2_op = SparsePauliOp.from_list(H2_op_list)

# Create an ansatz
ansatz = EfficientSU2(H2_op.num_qubits, reps=1)

# Decompose into individual gates
circuit = ansatz.decompose(reps=3)

circuit

/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(


In [2]:
# See how many parameters are in our ansatz
num_params = circuit.num_parameters
print(f'Num parameters: {num_params}')

# List out the observables
observables = H2_op.paulis
print(f'Observables: {observables}')

Num parameters: 8
Observables: ['II', 'IZ', 'XX', 'ZI', 'ZZ']


In [3]:
# Create a callback function that will be used to get VQE data during the run 
def callback(eval_count, parameters, mean, std):
    print(f"Round num: {eval_count}, energy: {mean}, parameters: {parameters}")

In [4]:
# Use the modified CutVQE to cut and run VQE for our problem
estimator = Estimator()
optimizer = COBYLA(maxiter=80)
vqe = CutVQE(estimator, circuit, optimizer, observables=observables, shots=2**12, max_subcircuit_width=1, max_cuts=9, num_subcircuits=[2], model='gurobi', num_samples=1500, callback=callback)
result = vqe.compute_minimum_eigenvalue(H2_op)
print(result)

Restricted license - for non-production use only - expires 2024-10-28
Set parameter TimeLimit to value 300
Set parameter Cutoff to value 1e+100
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (linux64)

CPU model: 11th Gen Intel(R) Core(TM) i7-11850H @ 2.50GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 31 rows, 21 columns and 74 nonzeros
Model fingerprint: 0xb35f4f20
Variable types: 0 continuous, 21 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e-02, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 1.0000000
Presolve removed 31 rows and 21 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.09 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 1: 1 

Optimal solution found (to

Round num: 47, energy: -1.8600349781511438, parameters: [ 2.95676233  6.57894343 -0.18546909  2.55918808 -2.32864929  0.23631629
 -5.49332709  5.6337079 ]
Round num: 48, energy: -1.863564024663336, parameters: [ 2.95466829  6.56084559 -0.18886466  2.57134634 -2.31605967  0.25791364
 -5.49945047  5.62552292]
Round num: 49, energy: -1.845497075880686, parameters: [ 2.94119485  6.56560799 -0.18608693  2.57544322 -2.34663783  0.26267075
 -5.50524132  5.61893734]
Round num: 50, energy: -1.8306066133363201, parameters: [ 2.96360957  6.55697039 -0.18255972  2.57991773 -2.32566371  0.24444025
 -5.50367975  5.62270147]
Round num: 51, energy: -1.8499337242425242, parameters: [ 2.95052843  6.56322853 -0.17795121  2.57838929 -2.32197352  0.24965295
 -5.50168682  5.62785503]
Round num: 52, energy: -1.8532089326437173, parameters: [ 2.95196452  6.55839033 -0.17137544  2.56696199 -2.3221369   0.25317039
 -5.49176789  5.61894713]
Round num: 53, energy: -1.8377126013746252, parameters: [ 2.95251665  6.

In [5]:
# Do the same thing to compare againt not cutting at all
estimator = Estimator()
optimizer = COBYLA(maxiter=80)
no_cut_vqe = VQE(ansatz=circuit, optimizer=optimizer, estimator=estimator, callback=callback)
no_cut_result = no_cut_vqe.compute_minimum_eigenvalue(H2_op)
print(no_cut_result)

Round num: 1, energy: -1.535582890339457, parameters: [3.5212242795789397, -5.411209281757448, 4.061122114930418, -2.519832344798718, -0.6180674824986241, 3.2704267446292405, 5.747696271124864, -1.1920541363239847]
Round num: 2, energy: -1.1930105220973082, parameters: [4.52122427957894, -5.411209281757448, 4.061122114930418, -2.519832344798718, -0.6180674824986241, 3.2704267446292405, 5.747696271124864, -1.1920541363239847]
Round num: 3, energy: -1.2043537290538082, parameters: [3.5212242795789397, -4.411209281757448, 4.061122114930418, -2.519832344798718, -0.6180674824986241, 3.2704267446292405, 5.747696271124864, -1.1920541363239847]
Round num: 4, energy: -1.4475907093830747, parameters: [3.5212242795789397, -5.411209281757448, 5.061122114930418, -2.519832344798718, -0.6180674824986241, 3.2704267446292405, 5.747696271124864, -1.1920541363239847]
Round num: 5, energy: -1.5276662110262562, parameters: [3.5212242795789397, -5.411209281757448, 4.061122114930418, -1.519832344798718, -0.6